# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset (adjust the path accordingly)
file_path = 'training.1600000.processed.noemoticon.csv'
df = pd.read_csv(file_path, encoding='latin-1', header=None)

# Define column names
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']

# Display the first few rows of the dataframe
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [2]:
# Sample a subset of the data
subset_df = df.sample(n=20000, random_state=42)

# Display the first few rows of the subset
subset_df.head()

,target,id,date,flag,user,text
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [3]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

# Define the clean_up() function
def clean_up(input_string):
    cleaned_string = re.sub(r'[^a-zA-Z\s]', '', input_string)
    cleaned_string = cleaned_string.replace("'", ' ')
    cleaned_string = re.sub(r'http\S+', '', cleaned_string)
    cleaned_string = re.sub(r's(?=q\s)', 's ', cleaned_string, flags=re.IGNORECASE)
    cleaned_string = re.sub(r'ironhacks', 'ironhack s', cleaned_string, flags=re.IGNORECASE)
    cleaned_string = re.sub(r'\s+', ' ', cleaned_string)
    return cleaned_string.lower().strip()

# Define the tokenize() function
def tokenize(input_string):
    words = [word[1:] if word.startswith("'") else word for word in word_tokenize(input_string)]
    return words

# Define the stem_and_lemmatize() function
def stem_and_lemmatize(word_list):
    # Initialize the Porter stemmer and WordNet lemmatizer
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    # Apply stemming and lemmatization to each word in the list
    stemmed_and_lemmatized_words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in word_list]
    return stemmed_and_lemmatized_words

def remove_stopwords(words):
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return filtered_words

# Assuming df is your original DataFrame
# If you don't have a DataFrame, you should load your data first

# Apply the clean_up() function
subset_df['cleaned_text'] = subset_df['text'].apply(clean_up)

# Apply the tokenize() function
subset_df['tokenized_text'] = subset_df['cleaned_text'].apply(tokenize)

# Apply the stem_and_lemmatize() function
subset_df['stem_lemma_text'] = subset_df['tokenized_text'].apply(stem_and_lemmatize)

# Apply the remove_stopwords() function
subset_df['text_processed'] = subset_df['stem_lemma_text'].apply(remove_stopwords)

# Display the first few rows of the dataframe with processed text
subset_df[['target', 'id', 'date', 'flag', 'user', 'text', 'text_processed']].head()

,target,id,date,flag,user,text,text_processed
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!,"[chrishasboob, ahhh, hope, ok]"
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo...","[misstoriblack, cool, tweet, app, razr]"
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...,"[tiannachao, know, famili, drama, lamehey, nex..."
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...,"[school, email, wont, open, geographi, stuff, ..."
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem,"[upper, airway, problem]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [4]:
from nltk import FreqDist

# Combine all words in text_processed
all_words = [word for sublist in subset_df['text_processed'] for word in sublist]

# Calculate the frequency distribution of all words
freq_distribution = FreqDist(all_words)

# Select the top 5,000 words
top_words = freq_distribution.most_common(5000)

# Extract only the words from the top_words list
top_words_list = [word[0] for word in top_words]

# Print the top 100 top_words
print(top_words[:100])

[('im', 2219), ('go', 1700), ('wa', 1348), ('get', 1344), ('day', 1263), ('good', 1144), ('thi', 1143), ('work', 1091), ('like', 1033), ('love', 1010), ('dont', 895), ('got', 859), ('today', 835), ('time', 810), ('u', 795), ('cant', 765), ('thank', 736), ('want', 726), ('one', 711), ('lol', 703), ('miss', 689), ('know', 679), ('back', 671), ('feel', 641), ('realli', 640), ('see', 630), ('think', 621), ('well', 597), ('watch', 582), ('hope', 581), ('still', 563), ('night', 544), ('make', 533), ('need', 523), ('amp', 518), ('look', 517), ('na', 511), ('come', 509), ('oh', 500), ('home', 500), ('new', 488), ('ha', 458), ('last', 457), ('much', 439), ('twitter', 433), ('great', 426), ('morn', 409), ('wait', 406), ('wish', 401), ('ill', 400), ('sleep', 397), ('follow', 396), ('tomorrow', 383), ('tri', 379), ('haha', 367), ('say', 357), ('sad', 354), ('week', 353), ('bad', 350), ('veri', 348), ('onli', 347), ('sorri', 343), ('would', 340), ('happi', 340), ('right', 339), ('whi', 334), ('fun'

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# Assuming subset_df is your DataFrame containing 'target' and 'text_processed' columns

# Convert the 'text_processed' column back to a list of strings
documents = [' '.join(words) for words in subset_df['text_processed']]

# Create a CountVectorizer using the top 100 words
vectorizer = CountVectorizer(max_features=100, binary=True)
features = vectorizer.fit_transform(documents)

# Convert the features to a DataFrame
features_df = pd.DataFrame(features.toarray(), columns=vectorizer.get_feature_names_out())

# Add the 'is_positive' column to the features DataFrame
features_df['is_positive'] = subset_df['target'] == 4

# Convert the features DataFrame to the required Python list format
feature_list = []
for index, row in features_df.iterrows():
    features_dict = {col: bool(row[col]) for col in features_df.columns[:-1]}
    label = bool(row['is_positive'])
    feature_list.append((features_dict, label))

# Display the converted feature list (optional)
print(feature_list[:30])

[({'amp': False, 'awesom': False, 'back': False, 'bad': False, 'bed': False, 'better': False, 'cant': False, 'come': False, 'could': False, 'day': False, 'didnt': False, 'dont': False, 'even': False, 'everyon': False, 'feel': False, 'final': False, 'first': False, 'follow': False, 'friend': False, 'fun': False, 'get': False, 'go': False, 'gon': False, 'good': False, 'got': False, 'great': False, 'guy': False, 'ha': False, 'haha': False, 'happi': False, 'hate': False, 'hi': False, 'home': False, 'hope': True, 'hour': False, 'ill': False, 'im': False, 'ive': False, 'know': False, 'last': False, 'let': False, 'like': False, 'littl': False, 'lol': False, 'look': False, 'love': False, 'make': False, 'miss': False, 'morn': False, 'much': False, 'na': False, 'need': False, 'new': False, 'next': False, 'nice': False, 'night': False, 'oh': False, 'one': False, 'onli': False, 'peopl': False, 'play': False, 'realli': False, 'right': False, 'sad': False, 'say': False, 'school': False, 'see': False

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [6]:
from nltk.classify import NaiveBayesClassifier
from sklearn.model_selection import train_test_split

# Split the feature set into training and test sets
train_set, test_set = train_test_split(feature_list, test_size=0.2, random_state=42)

# Train the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(train_set)

# Show the most informative features
classifier.show_most_informative_features(10)

Most Informative Features
                     bed = True            False : True   =      3.2 : 1.0
                    hour = True            False : True   =      3.0 : 1.0
                   great = True             True : False  =      2.6 : 1.0
                    morn = True             True : False  =      2.5 : 1.0
                    hope = True            False : True   =      2.4 : 1.0
                   thing = True            False : True   =      2.4 : 1.0
                tomorrow = True             True : False  =      2.3 : 1.0
                     tri = True             True : False  =      2.3 : 1.0
                  though = True            False : True   =      2.3 : 1.0
                    know = True            False : True   =      2.2 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [7]:
from nltk.classify import accuracy

# Test the classifier with the test set
accuracy_score = accuracy(classifier, test_set)

# Print the accuracy score
print(f'Accuracy: {accuracy_score:.4f}')

# Optional: You can also print the error rate
error_rate = 1 - accuracy_score
print(f'Error Rate: {error_rate:.4f}')


Accuracy: 0.9882
Error Rate: 0.0118


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [8]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [9]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [10]:
# your code here